In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.random import randn

In [2]:
file_name = "vec.txt"

count = 0
embedding_matrix = []

print(" \n word embedding in process ... \n ")

with open(file_name) as infile:
	for line in infile:
		if count == 0:
			a = line.strip()
			shape = list(map(int, a.split(" ")))
		else :
			a = line.strip()
			# print(a)
			vect = list(map(float, a.split(" ")))
			embedding_matrix.append(vect)

		count = count + 1



embedding_matrix = np.array(embedding_matrix)
print(" Word embedding end embedding matrix shape is : ", embedding_matrix.shape)


 
 word embedding in process ... 
 
 Word embedding end embedding matrix shape is :  (13100, 128)


In [3]:

 # Utily Fonction
    
def shuffle_split_data(X, y):
    split = np.random.rand(X.shape[0]) < 0.7

    X_Train = X[split]
    y_Train = y[split]
    X_Test =  X[~split]
    y_Test = y[~split]

    return X_Train, y_Train, X_Test, y_Test



def get_minibatch(X, y, minibatch_size, shuffle=True):
	minibatches = []
	for i in range(0, X.shape[0], minibatch_size):
		X_mini = X[i:i + minibatch_size]
		y_mini = y[i:i + minibatch_size]
		minibatches.append((X_mini, y_mini))
	return minibatches


def get_embedding_vector(x, embedding_matrix, embed_dim):
    sentences = []
    embedd_vector = np.array((1,embed_dim))
    for xt in x:
        sentences.append(embedding_matrix[xt])
    return sentences


def softmax( x, axis = -1):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=axis)

def binary_loss_entropy(label, y_pred):
    """Compute the loss entropy beetween between two binary probability distributions"""
    idx = np.argmax(label)
    loss = -np.log(y_pred[idx])
    return loss

def accuracy(n, y, y_pred):
    target = np.argmax(y[0])
    n += int(np.argmax(y_pred[0]) == target)
    return  n





In [15]:

print(" \n Reading data in process ... \n ")


url = "DynamicallyGeneratedHateDataset.csv"
df = pd.read_csv(url)
data = df[['text','label']]
data = data[data.label != "Neutral"]
Y = pd.get_dummies(data.label).values

X = []
with open("data.txt") as infile:
	for line in infile:
		a = line.strip()
		# print(a)
		v = list(map(int, a.split(" ")))
		X.append(v)


X = np.array(X)


try:
    X_train, Y_train, X_test, Y_test = shuffle_split_data(X,Y)
#     print("Successful shuffle_split")
except:
    print("shuffle_split Fail")

print(" ---- Reading data end ---- \n")
print(" Train Shape : ")
print(X_train.shape,Y_train.shape)
print(" \n Test Shape : ")
print(X_test.shape,Y_test.shape)


 
 Reading data in process ... 
 
 ---- Reading data end ---- 

 Train Shape : 
(28901, 50) (28901, 2)
 
 Test Shape : 
(12354, 50) (12354, 2)


In [18]:
# print("Loss : ",   round(loss/len(batches), 4) , " ||  Accuracy : ", round(acc/len(batches), 4)  )

In [11]:

hiden_size = 64
embed_dim = 128 
output_size = 2
class MyRnn():
    
    def __init__(self, num_units=64, input_size=128, output_size=2):
        super(MyRnn, self).__init__()
        self.num_units = num_units
        self.input_size = input_size
        self.output_size = output_size
 
    def build(self):
          
        # self.W_hx = Wxh.numpy() 
        self.W_hx = randn(embed_dim, hiden_size)/10

        # self.W_hh = Whh.numpy()
        self.W_hh = np.identity(hiden_size)

        # self.b_h = bh.numpy()
        self.b_h = np.zeros((hiden_size, ))

        # self.W_yh = Why.numpy()
        self.W_yh = randn(hiden_size, output_size)/10

        # self.b_y = by.numpy()
        self.b_y = np.zeros((output_size, ))

        self.weights = [ self.W_hx, self.W_hh, self.b_h, self.W_yh,  self.b_h,  self.b_y]
        
        self.dW_hx = np.zeros(self.W_hx.shape)
        self.dW_hh = np.zeros(self.W_hh.shape)
        self.db_h = np.zeros(self.b_h.shape)
        self.dW_yh = np.zeros(self.W_yh.shape)
        self.db_y = np.zeros(self.b_y.shape)

        self.last_input = None

        self.h_last = None

        self.nbr_correct = 0
    



    def forward(self, sequence):
        """ Like the forward phase """
        h = [] 
        self.h_last = np.zeros([1, self.num_units])
        h.append(self.h_last)
        x = get_embedding_vector(sequence, embedding_matrix, 128)
        self.last_input = x

        for xt in x:
            xt = np.expand_dims(xt, axis=0)
            h_t =  np.dot(xt, self.W_hx)  +  np.dot(self.h_last, self.W_hh)  + self.b_h     
            self.h_last = np.tanh(h_t)
            h.append(self.h_last)

        y = np.dot(self.h_last, self.W_yh) + self.b_y

        return  softmax(y) , h
    
    def back_forward(self, X, Y, y_hat, h, learning_rate=0.01): 
             
        loss = binary_loss_entropy(Y, y_hat[0])

         # dy = y_pred - label
        dy = y_hat - Y
         
        #dWhy = last_h.T * dy 
        dWhy = np.dot( self.h_last.T , dy)

        #dby = dy = y_pred - label
        dby = dy[0]

        # Firstly we Initialize dWhh, dWhx, and dbh to zero 
        dWhh =  np.zeros(self.W_hh.shape)
        dWhx =  np.zeros(self.W_hx.shape)
        dbh =  np.zeros(self.b_h.shape)

        dhnext = np.zeros(self.b_h.shape)

        # dWhh, dWhx, and dbh was computed over time via back propagation through time

        dh = np.matmul( dy , self.W_yh.T  )



        for t in reversed(range(len(X))):

          # dh = np.dot(Why.T, dy) + dhnext ----- à revoir au cas où ------


          #dhraw = (1 - hs[t] * hs[t]) * dh
              dhraw = (1 - np.power( h[t+1], 2 ))  * dh   
  
          # dbh += dhraw
              dbh += dhraw[0]

          # dWxh += np.dot(dhraw, xs[t].T)
              dWhx += (np.matmul(np.expand_dims(self.last_input[t], axis=1) , dhraw))   
          #dWhh += np.dot(dhraw, hs[t-1].T)
              dWhh += (np.matmul( h[t].T, dhraw)) 
          #dhnext = np.dot(Whh.T, dhraw)
              dh = np.matmul( dhraw, self.W_hh.T )

      # Just aggregate the derived weight for this batch
    
        derived = [dWhx*0.5, dWhh*0.5, dbh*0.5, dWhy*0.5, dby*0.5]

        self.dW_hx = self.dW_hx + dWhx
        self.dW_hh = self.dW_hh + dWhh
        self.db_h =  self.db_h  + dbh
        self.dW_yh = self.dW_yh + dWhy
        self.db_y =  self.db_y  + dby

        return loss,  derived
    
    

    def gradient(self,  n, learning_rate=0.01):
        
        "Update the parameter with SGD"
        self.W_hx = self.W_hx - learning_rate*self.dW_hx*(1/n)
        self.W_hh = self.W_hh - learning_rate*self.dW_hh*(1/n)
        self.W_yh = self.W_yh - learning_rate*self.dW_yh*(1/n)
        self.b_h =  self.b_h  - learning_rate*self.db_h*(1/n)
        self.b_y =  self.b_y  - learning_rate*self.db_y*(1/n)
        "Gradient update is finish for this batch we need to reinitialize the derived for other batch"
        self.dW_hx = np.zeros(self.W_hx.shape)
        self.dW_hh = np.zeros(self.W_hh.shape)
        self.db_h  = np.zeros(self.b_h.shape)
        self.dW_yh = np.zeros(self.W_yh.shape)
        self.db_y  = np.zeros(self.b_y.shape)



    
    def evaluate(self, x_test, y_test):
        
        Loss = 0
        y_pred = []
        
        for x in x_test:
            y , _ = self.forward(x_mb)
            y_pred.append(y)
            
        for (y_true, y_hat) in zip(y_test, y_pred):
            
            Loss = Loss + binary_loss_entropy(y_true, y_hat[0])
            
        return  (round(Loss/len(x_test), 4))
            
            
            
    



In [12]:
batches = get_minibatch(X_train[:2000], Y_train[:2000] ,1)

In [13]:
print("\n TRAINING PHASE IN PROCESS ... ")

myrnn = MyRnn()
myrnn.build()
epoch = 200
for iter in range(20):
    print("\n Start of epoch %d/200" % (iter+1,))
    Loss = 0
    for (x_mb, y_mb) in batches:
        loss = 0
        n = 0
        for a , b in zip(x_mb, y_mb):
            y_pred, h = myrnn.forward(a)
            l, g = myrnn.back_forward(a, b, y_pred, h)
            loss = loss + l
            n = n + 1
        loss = loss / n
        myrnn.gradient(n)

        Loss = Loss + loss
    print("Loss : ",   round(Loss/len(batches), 4)  )
#     print("Loss : ",   round(loss/len(batches), 4) , " ||  Accuracy : ", round(acc/len(batches), 4)  )





 TRAINING PHASE IN PROCESS ... 

 Start of epoch 1/200
Loss :  0.66

 Start of epoch 2/200
Loss :  0.61

 Start of epoch 3/200
Loss :  0.5967

 Start of epoch 4/200
Loss :  0.5888

 Start of epoch 5/200
Loss :  0.6792

 Start of epoch 6/200
Loss :  0.6651

 Start of epoch 7/200
Loss :  0.627

 Start of epoch 8/200
Loss :  0.594

 Start of epoch 9/200
Loss :  0.5774

 Start of epoch 10/200
Loss :  0.5482

 Start of epoch 11/200
Loss :  0.6077

 Start of epoch 12/200
Loss :  0.615

 Start of epoch 13/200
Loss :  0.5785

 Start of epoch 14/200
Loss :  0.5768

 Start of epoch 15/200


KeyboardInterrupt: 

In [27]:
myrnn = MyRnn()
myrnn.build()
loss_test = myrnn.evaluate(X_test, Y_test)
loss_test

0.3193

In [19]:
y_pred

array([[0.12509235, 0.87490765]])

In [24]:
len(batches)

2000

In [25]:
for x_mb, y_mb in batches :
    pass

In [26]:
x_mb[0]

array([13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099,
       13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099,
          24,  6877,  1881,  6474,   423, 11577,    14,  3682,  1368,
          89,   111,   261,  6252,   221,  2035,    36,   966,   724,
        2405,    86,   699,  8765,   546,    74,  1674,    24,   243,
        1630,    15,   545,  3603,   413])

In [27]:
get_embedding_vector(x_mb[0], embedding_matrix, 128)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
embedding_matrix[0]

array([ 0.14963913, -0.26754704,  0.15737614,  0.17928906, -0.03344075,
       -0.37010404, -0.08545362,  0.13982582, -0.08779539, -0.0231225 ,
        0.05763461,  0.01970007, -0.01256656,  0.08484232,  0.15404165,
        0.08170771, -0.13887173, -0.07997812, -0.14389268, -0.13953087,
        0.16541514,  0.24788944,  0.26629055, -0.20225663,  0.29906383,
        0.34322241, -0.09092203,  0.06427828, -0.0275959 , -0.18191765,
       -0.21912366, -0.0079031 ,  0.0883453 , -0.09015875, -0.09665185,
        0.14325652,  0.28888321,  0.06744371,  0.25939837, -0.08205373,
        0.05369937, -0.04399143, -0.23468989, -0.43514314,  0.04003573,
       -0.16875082, -0.1953536 , -0.06781182,  0.1562645 ,  0.08493797,
       -0.09834261,  0.20233338, -0.0756415 , -0.00678068,  0.01964584,
        0.1829329 ,  0.01323973, -0.17713504, -0.13865913,  0.22101901,
       -0.150313  , -0.05182702, -0.01565925,  0.0156415 ,  0.18443356,
       -0.03071907, -0.12664998,  0.12493239,  0.03650161, -0.04